In [ ]:
# 1.문서 내용을 읽어옴. / # 2. 문서를 쪼갬.
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter( # 긴 텍스트를 LLM이 처리하기 좋게끔 조각으로 쪼개주는 함수
    chunk_size=1500, # 한 조각의 최대 문자 수
    chunk_overlap=200, # 이전 조각과 겹치는 문자 수 (문맥 손실 방지)
)

loader = Docx2txtLoader("./tax.docx") # 해당 문서를 읽기 위한 로더 객체를 생성함.
document_list = loader.load_and_split(text_splitter=text_splitter) 

In [3]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
#3.임베딩하여 데이터베이스에 저장함.

from langchain_chroma import Chroma # 로컬 기반 데이터베이스

# 데이터베이스를 처음 생성함.
# database = Chroma.from_documents(documents=document_list, 
#                                 embedding=embedding, # 문서에 임베딩을 적용하여 벡터로 변환함.
#                                 persist_directory="./chroma", # 영구 저장을 위한 데이터베이스 폴더를 생성함.
#                                 collection_name="chroma-tax", # 컬렉션을 생성함.                        
#                                 )

# 기존에 생성한 데이터베이스를 불러옴.
database = Chroma(persist_directory="./chroma",
                 collection_name="chroma-tax",
                 embedding_function=embedding) # 사용자의 질문(쿼리)에 임베딩을 적용하여 벡터로 변환함.

In [ ]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# 4.질문에 대한 유사도를 검색함.
retrieved_docs = database.similarity_search(query, k=3) # 질문(쿼리)을 임베딩한 후, 데이터베이스에 저장된 모든 문서 조각들과 유사도를 비교한 후, 유사도가 가장 높은 k개만 반환함.

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [ ]:
# 5.유사도 검색 문서와 질문을 LLM에게 전달함.

prompt = f"""[identity]
    - 당신은 최고의 한국 소득세 전문가 입니다
    - [Context]를 참고해서 사용자의 질문에 답변해주세요
    
    [Context]
    {retrieved_docs}
    
    Question: {query}
"""

In [116]:
ai_message = llm.invoke(prompt)

In [117]:
ai_message.content

'한국의 소득세는 여러 요인들에 따라 결정되며, 가장 중요한 요인은 과세표준입니다. 연봉 5천만 원인 직장인의 경우, 기본적인 소득세 부담을 계산하기 위해서는 먼저 공제 항목들을 고려해야 합니다. \n\n대부분의 경우, 연금보험료, 건강보험료, 고용보험료, 그리고 근로소득공제 등을 차감한 후 과세표준이 결정됩니다. 하지만 기본적인 과세표준을 고려하지 않고 단순히 연봉만으로 소득세를 계산한다면, 여기서는 한국의 기본 소득세율을 기준으로 대략적인 추정만 가능합니다.\n\n2023년 기준으로, 한국의 소득세율은 과세표준에 따라 달라집니다. 5천만 원의 경우, 과세표준이 1,200만 원 초과 ~ 4,600만 원 이하 구간에 속하므로, 15%의 세율이 적용됩니다. 따라서, 기본공제 등을 고려하지 않았을 때, 단순히 5천만 원에 대해 15%를 적용하면 약 750만 원이 소득세가 될 수 있습니다. \n\n그러나 실제 소득세는 개인의 공제 항목과 세액 공제를 고려한 후 계산되므로, 정확한 금액을 알고 싶다면 국세청의 소득세 계산기를 사용하는 것이 좋습니다. 또한, 근로소득세 공제나 추가적인 공제 항목들이 많으므로, 실제 부과되는 세금은 이보다 낮아질 수 있습니다.'

### RetrievalQA 활용

In [5]:
from langchain_classic import hub

prompt = hub.pull("rlm/rag-prompt") # RAG용 표준 프롬프트를 LangChain Hub에서 가져옴. ("작성자/프롬프트명")

In [ ]:
from langchain_classic.chains import RetrievalQA 

qa_chain = RetrievalQA.from_chain_type( # 벡터 검색, 프롬프트, LLM을 하나로 묶은 RAG 체인을 생성함. 
    retriever = database.as_retriever(search_kwargs={"k": 3}), # 데이터베이스를 리트리버 인터페이스로 변환함. / # 4.질문에 대한 유사도를 검색함.
    chain_type_kwargs = {"prompt": prompt}, # 사용자 지정 프롬프트로 교체함.
    llm = llm # 5.유사도 검색 문서와 질문을 LLM에게 전달함.
)


ai_message = qa_chain({"query": query})